<a href="https://colab.research.google.com/github/LucianoBV/Procesamiento-del-habla/blob/main/TEMA_1_DETECCI%C3%93N_DE_ANOMAL%C3%8DAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESAFÍO OBLIGATORIO FINAL MARTÍN FIERRO
**MATERIA:** Procesamiento del habla

**DOCENTE:** Ingeniera en Sistemas de Información Ana Laura Diedrichs

**ALUMNO:** Vargas Luciano Benjamín



# PRESENTACION DEL DATA SET
Para este proyecto vamos a utilizar los libros gauchescos EL Martín Fierro y Fausto (de Estanislao Lopez).


### Descargamos los archivos desde la url

In [ ]:
!pip install pdfplumber

In [ ]:
import requests
import os
import pdfplumber

In [ ]:
#creo la variable url
url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"

#hago el get y guardo la respuesta
response = requests.get(url)

#si la respuesta es ok, abro el archivo, descargo los datos y con eso escribo el archivo
if response.status_code == 200:
  Martin = "el_gaucho_martin_fierro.pdf"
  with open(Martin, "wb") as file:
    file.write(response.content)
#imprimo el peso para verificar que se haya descargado correctamente
  file_size = os.path.getsize(Martin)
  print(f"PDF descargado exitosamente, tamaño del archivo: {file_size} bytes")
else:
#veo cual es el código de estado del response si la respuesta no fue 200
  print(f"Error al descargar el PDF. Código de estado: {response.status_code}")

PDF descargado exitosamente, tamaño del archivo: 513796 bytes


In [ ]:
#creo la variable url
url = "https://www.gba.gob.ar/sites/default/files/cultura/archivos/FAUSTO-Estanislado_del_Campo.pdf"

#hago el get y guardo la respuesta
response = requests.get(url)

#si la respuesta es ok, abro el archivo, descargo los datos y con eso escribo el archivo
if response.status_code == 200:
  Fausto = "FAUSTO-Estanislado_del_Campo.pdf"
  with open(Fausto, "wb") as file:
    file.write(response.content)
#imprimo el peso para verificar que se haya descargado correctamente
  file_size = os.path.getsize(Fausto)
  print(f"PDF descargado exitosamente, tamaño del archivo: {file_size} bytes")
else:
#veo cual es el código de estado del response si la respuesta no fue 200
  print(f"Error al descargar el PDF. Código de estado: {response.status_code}")

PDF descargado exitosamente, tamaño del archivo: 157971 bytes


# PREPROCESAMIENTO DE LOS TEXTOS

### Función para verificar cada página del libro y excluirla si no es texto específico del libro

In [ ]:
def pagina_a_procesar(texto_pagina):
    # Palabras clave para omitir ciertas secciones como índice, prólogo, anexos, etc.
    palabras_omitir = ['Índice', 'Prólogo', 'Anexo', 'Imágenes', 'Agradecimientos', 'Dedicatoria', 'http',]

    # Revisar si la página contiene alguna palabra que indique que se excluirá
    for palabra in palabras_omitir:
        if palabra.lower() in texto_pagina.lower():
            return False
    return True

# Función para extraer el texto importante del PDF
def extraer_texto(filename):
    texto_relevante = ''

    with pdfplumber.open(filename) as pdf:
        pagina_inicio=0
        pagina_fin=len(pdf.pages)
        # Iterar sobre las páginas en el rango seleccionado
        for pagina_numero in range(pagina_inicio, pagina_fin):
            pagina = pdf.pages[pagina_numero]
            texto_pagina = pagina.extract_text()

            # Si la página no tiene texto o es irrelevante, omitirla
            if texto_pagina and pagina_a_procesar(texto_pagina):
                texto_relevante += texto_pagina + '\n'

    return texto_relevante

 # Extraer el texto relevante del rango de páginas
Martin_Fierro = extraer_texto(Martin)
Fausto_Estanislao = extraer_texto(Fausto)


In [ ]:
# Mostrar el texto extraído
print("Texto relevante extraído:")
print(Martin_Fierro)

In [ ]:
# Mostrar el texto extraído
print("Texto relevante extraído:")
print(Fausto_Estanislao)

### Función para dividir el texto en estrofas, las limpia y nos devuelve una lista estructurada que representa el corpus del poema.

In [ ]:
# Importamos el  módulo re, que proporciona soporte para expresiones regulares para la búsqueda de patrones dentro del texto.
import re

def crear_corpus(texto):
    # Reemplazar puntos finales y signos de exclamación por doble espacio para separar estrofas
    texto_modificado = re.sub(r'([.!])', r'\1  ', texto)

    # Dividir el texto en estrofas usando doble espacio como delimitador
    estrofas = texto_modificado.split('  ')

    # Limpiar estrofas vacías y eliminar espacios adicionales
    estrofas = [estrofa.strip() for estrofa in estrofas if estrofa.strip()]

    return estrofas

In [ ]:
# Crear el corpus de documentos
corpus_documentos_Martin = crear_corpus(Martin_Fierro)
corpus_documentos_Fausto = crear_corpus(Fausto_Estanislao)

In [ ]:
cantidad_estrofas = len(corpus_documentos_Martin)
print(f"El corpus 'corpus_documentos_Martin' contiene {cantidad_estrofas} estrofas del Martín Fierro.")

In [ ]:
cantidad_estrofas = len(corpus_documentos_Fausto)
print(f"El corpus 'corpus_documentos_Fausto' contiene {cantidad_estrofas} estrofas del Fausto.")

In [ ]:
# Imprimir el corpus para verificar los resultados
for idx, documento in enumerate(corpus_documentos_Martin):
    print(f"Documento {idx+1}: {documento}")

In [ ]:
# Imprimir el corpus para verificar los resultados
for idx, documento in enumerate(corpus_documentos_Fausto):
    print(f"Documento {idx+1}: {documento}")

# TEMA 1 DETECCIÓN DE ANOMALÍAS

El objetivo es entrenar un modelo para que distinga si el texto es o no parte del Martín Fierro.
Para ello debe entrenar un modelo de embeddings para el texto de Martín Fierro.
Luego comprobar que usando dicho modelo de embeddings con otros textos,usando la distancia coseno, por ejemplo. Podría revisar otras medidas de distancias. Contrastar con otro libro de poesía gauchesca como Fausto (de Estanislao Lopez).


In [ ]:
# Importamos librerias necesarias
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split

### Tokenización de los textos

In [ ]:
# Tokenización para que el modelo entienda el texto
martin_fierro_tokens = [line.split() for line in corpus_documentos_Martin if line]
fausto_tokens = [line.split() for line in corpus_documentos_Fausto if line]

In [ ]:
print(martin_fierro_tokens)

In [ ]:
print(fausto_tokens)

### Modelo de embeding elegido Word2Vec

In [ ]:
# Entrenamos el modelo Word2Vec
modelo_w2v = Word2Vec(sentences=martin_fierro_tokens, vector_size=100, window=5, min_count=1, workers=4)

### Función para obtener un vector

In [ ]:
# Creamos esta función para obtener un vector que capture el significado del texto completo.
# Lo hacemos utilizando el modelo Word2Vec pre-entrenado.
def representar_texto(texto, modelo):
    vectores = [modelo.wv[word] for word in texto.split() if word in modelo.wv]
    return np.mean(vectores, axis=0) if vectores else np.zeros(modelo.vector_size)

In [ ]:
# Representaciones de los dos textos completos
vector_martin_fierro = representar_texto(' '.join([' '.join(line) for line in martin_fierro_tokens]), modelo_w2v)
vector_fausto = representar_texto(' '.join([' '.join(line) for line in fausto_tokens]), modelo_w2v)

In [ ]:
print(vector_martin_fierro)

### Comparación usando distancia coseno y resultados

In [ ]:
similitud_coseno = cosine_similarity([vector_martin_fierro], [vector_fausto])

In [ ]:
print(f"Similitud entre Martín Fierro y Fausto: {similitud_coseno[0][0]:.4f}")

Comparamos qué tan similar es el texto de "Martín Fierro" al texto de "Fausto" basándonos en sus representaciones numéricas usando la similitud coseno que es una forma de medir qué tan parecidos son dos vectores en términos de su dirección.
Como podemos ver los resultados muestran que los dos textos son muy parecidos.

### Comparacion usando la distancia eucleciana y resultados

In [ ]:
from scipy.spatial.distance import euclidean

# Calcular la distancia euclidiana entre los vectores
distancia = euclidean(vector_martin_fierro, vector_fausto)

# Imprimir la distancia
print(f"Distancia euclidiana entre Martín Fierro y Fausto: {distancia:.4f}")

A diferencia de la similitud coseno que se centra en la orientación de los vectores, la distancia euclidiana se centra en la magnitud de la diferencia entre ellos.
Como observamos en los resultados es muy baja la distancia indicando que son muy parecidos los textos.


Explicación de los Cambios:

Definir obtener_embedding_estrofa: El código ahora incluye la definición de la función obtener_embedding_estrofa. Esta función toma una cadena de texto (la estrofa) y el modelo Word2Vec como entrada, y devuelve el vector de embedding para esa estrofa.

Renombrar modelo a modelo_w2v: Para mayor claridad y consistencia, la variable modelo (que presumiblemente se refiere a tu modelo Word2Vec) ha sido renombrada a modelo_w2v.

Calcular embeddings_train: El código ahora incluye un marcador de posición para calcular embeddings_train. Reemplaza este marcador de posición con tu lógica real para calcular los embeddings para los datos de entrenamiento.

Usar modelo_w2v en las Llamadas a la Función: Las llamadas a obtener_embedding_estrofa se han actualizado para usar modelo_w2v como el argumento del modelo.

### Comparacion de una estrofa del Martin Fierro versus otra estrofa del Fausto

In [ ]:
# Define la función obtener_embedding_estrofa
def obtener_embedding_estrofa(estrofa, modelo):
    """
    Obtiene el embedding (representación vectorial) de una estrofa utilizando un modelo Word2Vec.

    Args:
        estrofa (str): La estrofa de texto.
        modelo (Word2Vec): El modelo Word2Vec pre-entrenado.

    Returns:
        numpy.ndarray: El embedding de la estrofa.
    """
    tokens = estrofa.split()
    vectores = [modelo.wv[token] for token in tokens if token in modelo.wv]
    return np.mean(vectores, axis=0) if vectores else np.zeros(modelo.vector_size)

In [ ]:
# 1. Seleccionar ejemplos de estrofas
estrofa_martin_fierro = corpus_documentos_Martin[10]  # Ejemplo de estrofa del Martín Fierro
estrofa_fausto = corpus_documentos_Fausto[5]  # Ejemplo de estrofa del Fausto

In [ ]:
# 2. Calcular embeddings de las estrofas
embedding_mf = obtener_embedding_estrofa(estrofa_martin_fierro, modelo_w2v)
embedding_fausto = obtener_embedding_estrofa(estrofa_fausto, modelo_w2v)

In [ ]:
# 3. Calcular embedding promedio del Martín Fierro
embeddings_train = [obtener_embedding_estrofa(estrofa, modelo_w2v) for estrofa in corpus_documentos_Martin]
embedding_promedio_mf = np.mean(embeddings_train, axis=0)

In [ ]:
# 4. Calcular distancias con cosine_similarity
distancia_mf_promedio = cosine_similarity([embedding_mf], [embedding_promedio_mf])[0][0]
distancia_fausto_promedio = cosine_similarity([embedding_fausto], [embedding_promedio_mf])[0][0]

In [ ]:
# 5. Imprimir resultados
print(f"Estrofa Martín Fierro: {estrofa_martin_fierro}")
print(f"Estrofa Fausto: {estrofa_fausto}")
print(f"Distancia estrofa MF a promedio MF: {distancia_mf_promedio:.4f}")
print(f"Distancia estrofa Fausto a promedio MF: {distancia_fausto_promedio:.4f}")

In [ ]:
# Calcula la distancia euclidiana
distancia = euclidean(embedding_mf, embedding_fausto)

# Imprime la distancia
print(f"Distancia euclidiana entre la estrofa del Martín Fierro y la estrofa del Fausto: {distancia:.4f}")

### Explicación:

**Seleccionar ejemplos de estrofas:**

Elegimos ejemplos de estrofas del Martín Fierro y del Fausto utilizando los corpus creados previamente.

**Calcular embeddings de las estrofas:**

Se utiliza la función obtener_embedding_estrofa para calcular los embeddings de las estrofas seleccionadas.

**Calcular embedding promedio del Martín Fierro:**

Se utiliza el embedding promedio del Martín Fierro, calculado previamente como embedding_promedio_mf, como referencia.

**Calcular distancias:**

Se utiliza la distancia coseno y euclediana para calcular la similitud entre los embeddings de las estrofas y el embedding promedio del Martín Fierro.

**Imprimir resultados:** Se muestran las estrofas seleccionadas, la distancia de la estrofa del Martín Fierro al promedio del Martín Fierro y la distancia de la estrofa del Fausto al promedio del Martín Fierro.



### **Conclusión:**


El análisis realizado empleando Word2Vec para obtener embeddings de las estrofas del Martín Fierro y el Fausto, junto con la aplicación de la similitud coseno y la distancia euclidiana, ha revelado una alta similitud entre las obras.

Similitud Coseno: Los resultados de la similitud coseno indican una fuerte relación entre los vectores que representan las estrofas de ambas obras, lo que sugiere una notable semejanza en su estilo y vocabulario.

Distancia Euclidiana: La distancia euclidiana, por otro lado, también ha arrojado valores bajos, confirmando la proximidad entre las estrofas en términos de su representación numérica.

Contexto: Esta similitud puede ser atribuida al hecho de que ambas obras pertenecen al género de la poesía gauchesca, compartiendo características lingüísticas y temáticas propias de la cultura y el habla popular de la región pampeana argentina.



#Conclusión Final:

El análisis realizado con Word2Vec, la similitud coseno y la distancia euclidiana proporciona evidencia clara y sólida de la notable similitud entre el Martín Fierro y el Fausto en términos de su estilo y vocabulario gauchesco.